In [2]:
from pyngrok import ngrok
import pandas as pd
import random
import plotly.express as px
from dash import Dash, dcc, html
import os

In [3]:
# Step 1: Generate a detailed dataset and save it as CSV
data = {
    "Month": pd.date_range(start="2024-01-01", periods=12, freq="M").strftime("%b-%Y"),
    "Expense Category": random.choices(["Operations", "Marketing", "Research", "IT"], k=12),
    "Amount": [random.randint(10000, 50000) for _ in range(12)],
    "Region": random.choices(["North", "South", "East", "West"], k=12),
    "Sales": [random.randint(50000, 150000) for _ in range(12)],
    "Type": random.choices(["AP", "AR"], k=12),
    "Aging Bucket": random.choices(["0-30 days", "31-60 days", "61-90 days", "90+ days"], k=12),
}


<ipython-input-3-4d8ceef1d425>:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "Month": pd.date_range(start="2024-01-01", periods=12, freq="M").strftime("%b-%Y"),


In [4]:
print(data)

{'Month': Index(['Jan-2024', 'Feb-2024', 'Mar-2024', 'Apr-2024', 'May-2024', 'Jun-2024',
       'Jul-2024', 'Aug-2024', 'Sep-2024', 'Oct-2024', 'Nov-2024', 'Dec-2024'],
      dtype='object'), 'Expense Category': ['Operations', 'IT', 'Marketing', 'IT', 'Marketing', 'Marketing', 'IT', 'Research', 'Marketing', 'Operations', 'Operations', 'Marketing'], 'Amount': [28076, 46737, 30352, 11139, 32367, 18160, 33246, 42226, 30198, 41791, 30185, 14739], 'Region': ['West', 'North', 'South', 'West', 'South', 'East', 'South', 'West', 'North', 'West', 'South', 'South'], 'Sales': [56913, 128149, 97406, 87086, 130376, 58370, 143787, 72087, 81238, 122990, 61064, 141767], 'Type': ['AR', 'AP', 'AP', 'AP', 'AR', 'AP', 'AR', 'AP', 'AR', 'AR', 'AP', 'AR'], 'Aging Bucket': ['61-90 days', '90+ days', '0-30 days', '31-60 days', '0-30 days', '0-30 days', '61-90 days', '0-30 days', '31-60 days', '0-30 days', '61-90 days', '61-90 days']}


In [6]:
# Save the data to a CSV file
csv_file = "financial_data.csv"
df = pd.DataFrame(data)
df.to_csv(csv_file, index=False)

In [7]:
# Step 2: Load the dataset from CSV
df = pd.read_csv(csv_file)

In [8]:
# Step 3: Create detailed and visually appealing graphs

# Expense Summary
expense_summary = df.groupby("Expense Category")["Amount"].sum().reset_index()
expense_chart = px.bar(
    expense_summary,
    x="Expense Category",
    y="Amount",
    title="Expense Summary",
    color="Expense Category",
    text="Amount",
    template="plotly_dark",
)
expense_chart.update_traces(texttemplate='%{text:.2s}', textposition='outside')
expense_chart.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

In [10]:
expense_chart.update_traces(texttemplate='%{text:.2s}', textposition='outside')
expense_chart.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

# Sales by Region
sales_summary = df.groupby("Region")["Sales"].sum().reset_index()
sales_chart = px.pie(
    sales_summary,
    names="Region",
    values="Sales",
    title="Sales by Region",
    hole=0.3,
    template="plotly_dark",
)

In [11]:
# AP and AR Aging Summary
ap_ar_summary = df.groupby(["Type", "Aging Bucket"])["Amount"].sum().reset_index()
aging_chart = px.bar(
    ap_ar_summary,
    x="Aging Bucket",
    y="Amount",
    color="Type",
    barmode="group",
    title="AP and AR Aging Summary",
    text="Amount",
    template="plotly_dark",
)
aging_chart.update_traces(texttemplate='%{text:.2s}', textposition='outside')

In [14]:
# Step 4: Set up the Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1("CFO Dashboard", style={"textAlign": "center", "color": "white"}),

    html.Div([
        html.H2("Expense Summary", style={"color": "white"}),
        dcc.Graph(figure=expense_chart)
    ], style={"marginBottom": "30px"}),

    html.Div([
        html.H2("Sales by Region", style={"color": "white"}),
        dcc.Graph(figure=sales_chart)
    ], style={"marginBottom": "30px"}),

    html.Div([
        html.H2("AP and AR Aging Summary", style={"color": "white"}),
        dcc.Graph(figure=aging_chart)
    ], style={"marginBottom": "30px"})
], style={"backgroundColor": "#111111", "padding": "20px"})

# Step 5: Run the Dash server with ngrok
public_url = ngrok.connect(8050)
print(f"Dash app running at: {public_url}")
app.run_server(debug=True, port=8050)

Dash app running at: NgrokTunnel: "https://fd44-34-48-18-108.ngrok-free.app" -> "http://localhost:8050"


<IPython.core.display.Javascript object>